In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!git clone https://github.com/ahmadelsallab/MultiCheXNet.git

In [ ]:
import pandas as pd 
import cv2
import numpy as np
import os
from glob import glob
import math
import matplotlib.pyplot as plt

import re
import html
import string
import unicodedata
from nltk.tokenize import word_tokenize

In [ ]:
! conda install -y gdown

In [ ]:
df  =pd.read_csv("../input/chest-xrays-indiana-university/indiana_reports.csv")
df.head(30)

In [ ]:
len(df[df['Problems'] == 'normal'])/len(df)

In [ ]:
df['findings'].iloc[0:10].tolist()

In [ ]:
print(df.shape)
df['impression'].unique().shape

In [ ]:
df['MeSH'].unique().tolist()[:10]

In [ ]:
img = cv2.imread('../input/chest-xrays-indiana-university/images/images_normalized/1004_IM-0005-1001.dcm.png')
plt.imshow(img)
plt.show()

In [ ]:
df2 = pd.read_csv("/kaggle/input/chest-xrays-indiana-university/indiana_projections.csv")
df2.head()

In [ ]:
df2.projection.unique()

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow

In [ ]:
def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x1))


def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')


def to_lowercase(text):
    return text.lower()

def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):
    """
    :param words:
    :type words:
    :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
    or
    from spacy.lang.en.stop_words import STOP_WORDS
    :type stop_words:
    :return:
    :rtype:
    """
    return [word for word in words if word not in stop_words]


def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def text2words(text):
    return word_tokenize(text)

def normalize_text( text):
    text = remove_special_chars(text)
    text = remove_non_ascii(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    #words = text2words(text)
    #stop_words = stopwords.words('english')
    #words = remove_stopwords(words, stop_words)
    #words = stem_words(words)# Either stem ovocar lemmatize
    #words = lemmatize_words(words)
    #words = lemmatize_verbs(words)

    return text
  
def normalize_corpus(corpus):
    return [normalize_text(t) for t in corpus]
  

In [ ]:
df.isna().sum()

In [ ]:
df= df.dropna(subset=['findings'])

In [ ]:
%time
df['findings_cleaned'] = df['findings'].apply(normalize_text)

In [ ]:
df['findings_cleaned'] = 'startseq '+df['findings_cleaned']+' endseq'

In [ ]:
for i in range(30):
    print(df.findings_cleaned.iloc[i])
    print('=========================================')

In [ ]:
num_words = []
for row in df['findings_cleaned'].tolist():
    num_words.append(len(word_tokenize(row)))

In [ ]:
num_words= np.array(num_words)
print("min length             : ", num_words.min())
print("max length             : ", num_words.max())
print("50th percentile length : ", np.percentile(num_words,50))
print("75th percentile length : ", np.percentile(num_words,75))
print("90th percentile length : ", np.percentile(num_words,90))
print("95th percentile length : ", np.percentile(num_words,95))
print("98th percentile length : ", np.percentile(num_words,98))
print("98th percentile length : ", np.percentile(num_words,99))

In [ ]:
vocab_size = 10000
max_len = 100

tok = Tokenizer(num_words=vocab_size,  oov_token='UNK' )
tok.fit_on_texts(df['findings_cleaned'].tolist())

vocab_size = len(tok.word_index) + 1
vocab_size

In [ ]:
from MultiCheXNet.data_loader.indiana_dataloader import get_train_validation_generator

In [ ]:
max_vocab_size=10000
max_len=100

csv_path1  ="/kaggle/input/chest-xrays-indiana-university/indiana_reports.csv"
csv_path2  ="/kaggle/input/chest-xrays-indiana-university/indiana_projections.csv"
img_path   ="/kaggle/input/chest-xrays-indiana-university/images/images_normalized/"

train_dataloader, val_dataloader, vocab_size, tok = get_train_validation_generator(csv_path1,csv_path2,img_path, max_vocab_size,max_len,hist_eq=True,normalize=True,augmentation=True)

In [ ]:
X,Y = next(enumerate(train_dataloader))[1]

In [ ]:
plt.imshow(X[0][0])
plt.show()

In [ ]:
from MultiCheXNet.evaluation.report_gen_evaluation import evaluate_from_dataloader

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam

In [ ]:
## Input layers
img_input = layers.Input(shape= (256,256,3)) 
report_input= layers.Input(shape= (max_len-1,))

## Encoder ######################

Densenet_model = tf.keras.applications.DenseNet121(
            include_top=False,
            weights="imagenet",
            input_shape=(256,256,3),
        )
number_of_encoder_layers=  len(Densenet_model.layers)

encoder_output = Densenet_model(img_input)
encoder_output = layers.Flatten()(encoder_output)
encoder_output = layers.Dropout(0.2)(encoder_output)
encoder_output = layers.Dense(512,activation='relu')(encoder_output)

##decoder ########################

#layers
gru_layer =  layers.GRU(512, return_sequences=True)
dense_layer= layers.Dense(vocab_size,activation='softmax')
embedding_layer = layers.Embedding(vocab_size, 300, mask_zero=True)
dropout = layers.Dropout(0.2)

# decoder model
embedding_output = embedding_layer(report_input)
gru_output = gru_layer(embedding_output, initial_state=encoder_output )
gru_output = dropout(gru_output)
output  = dense_layer(gru_output)
model = Model([img_input,report_input ],output)

##Inference models ################

#encoder_inference model
encoder_model = Model(img_input,encoder_output)

#decoder_inference model
prev_hidden_state= layers.Input(shape= (512))
report_input2 = layers.Input(shape= (1,))
embedding_output2= embedding_layer(report_input2)
gru_output2 = gru_layer(embedding_output2, initial_state=prev_hidden_state )
gru_output2 = dropout(gru_output2)
output2 = dense_layer(gru_output2)
decoder_model = Model([report_input2,prev_hidden_state],[output2,gru_output2])

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes=True)

In [ ]:
epochs =5
lr=1e-3
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr))

In [ ]:
hist = model.fit_generator( train_dataloader,
                    validation_data = val_dataloader,
                    epochs = epochs
                )

In [ ]:
y= hist.history['loss']
y_val = hist.history['val_loss']

x = range(0,len(y))
plt.plot(x,y)
plt.plot(x,y_val)
plt.show()

In [ ]:
evaluate_from_dataloader(val_dataloader,tok,encoder_model,decoder_model,max_len, decoder_type="GRU")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K

In [ ]:
## Input layers
img_input = layers.Input(shape= (256,256,3)) 
report_input= layers.Input(shape= (max_len-1,))

## Encoder ######################

Inception_model = tf.keras.applications.InceptionV3(
            include_top=False,
            weights=None,#"imagenet",
            input_shape=(256,256,3),
        )
number_of_encoder_layers=  len(Inception_model.layers)

encoder_output = Inception_model(img_input)
encoder_output = layers.Flatten()(encoder_output)

X_img = layers.Dropout(0.5)(encoder_output)
X_img = layers.Dense(300, use_bias = False, 
                        kernel_regularizer=regularizers.l2(1e-4),
                        name = 'dense_img')(X_img)
X_img = layers.BatchNormalization(name='batch_normalization_img')(X_img)
X_img = layers.Lambda(lambda x : K.expand_dims(x, axis=1))(X_img)

##decoder ########################

X_text = layers.Embedding(vocab_size, 300, mask_zero = True, name = 'emb_text')(report_input)
X_text = layers.Dropout(0.5)(X_text)

# Initial States


LSTMLayer = layers.LSTM(300, return_sequences = True, return_state = True, dropout=0.5, name = 'lstm')

# Take image embedding as the first input to LSTM
_, a, c = LSTMLayer(X_img)

A, _, _ = LSTMLayer(X_text, initial_state=[a, c])
output = layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax',
                                 kernel_regularizer = regularizers.l2(1e-4), 
                                 bias_regularizer = regularizers.l2(1e-4)), name = 'time_distributed_softmax')(A)




model  = Model(inputs=[img_input, report_input], outputs=output, name='NIC_greedy_inference_v2')


##Inference models ################

#encoder_inference model
encoder_model = Model(img_input,[a,c])

# Decoder model ###################

a0 = layers.Input(shape=(300,))
c0 = layers.Input(shape=(300,))

A, alast, clast = LSTMLayer(X_text, initial_state=[a0, c0])
output = layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax',
                                 kernel_regularizer = regularizers.l2(1e-4), 
                                 bias_regularizer = regularizers.l2(1e-4)), name = 'time_distributed_softmax')(A)



decoder_model = Model([report_input,a0,c0],[output,alast,clast])

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes=True)

In [ ]:
epochs =5
lr=1e-4
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr))

In [ ]:
hist = model.fit_generator( train_dataloader,
                    validation_data = val_dataloader,
                    epochs = epochs
                )

In [ ]:
y= hist.history['loss']
y_val = hist.history['val_loss']

x = range(0,len(y))
plt.plot(x,y)
plt.plot(x,y_val)
plt.show()

In [ ]:
from MultiCheXNet.evaluation.report_gen_evaluation import get_predictions_from_data_loader
from copy import deepcopy

val_dataloader_tmp = deepcopy(val_dataloader)
val_dataloader_tmp.nb_iteration  = 3
GT , preds = get_predictions_from_data_loader(val_dataloader_tmp,tok,encoder_model, decoder_model,max_len,decoder_type='LSTM')

In [ ]:
for index in range(8):
    print(GT[index])
    print(("====================================="))
    print(preds[index])
    print("")
    print("")

In [ ]:
# Inception predections
"""
the heart size is stable the aorta is ectatic and atherosclerotic but stable xxxx sternotomy xxxx are again noted the scarring in the left lower lobe is again noted and unchanged from prior exam there are mild bilateral prominent lung interstitial opacities consistent with emphysematous disease the calcified granulomas are stable
=====================================
humeral sequela sequela sequela denser denser denser denser denser denser denser denser sequela denser humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral


chest no focal consolidation no visualized pneumothorax no large pleural effusions the heart size is normal no focal thoracic bony findings hand there is also cortical and trabecular irregularity through the xxxx of the scaphoid there is a small cortical lucency through the base of the fourth metacarpal that may be a vascular xxxx
=====================================
humeral sequela sequela sequela denser denser denser denser denser denser denser denser sequela denser humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral


the cardiomediastinal silhouette is normal size and configuration pulmonary vasculature within normal limits there is a right lower lobe pneumonia no pleural effusion no pneumothorax
=====================================
humeral sequela sequela sequela denser denser denser denser denser denser denser sequela denser humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral


cardiac and mediastinal contours are within normal limits the lungs are clear bony structures are intact
=====================================
humeral sequela sequela sequela denser denser denser denser denser denser denser sequela denser humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral


heart size near top normal bilateral hilar fullness nonspecific in appearance mild aortic ectasiatortuosity diaphragm flattening and relative apical lucencies suggestive of emphysema xxxx and irregular interstitial markings right greater than left prominent left epicardial fat xxxx no focal alveolar consolidation no definite pleural effusion seen atrial septal occluder artifact mild spine curvature
=====================================
humeral sequela sequela sequela denser denser denser denser denser denser denser denser sequela denser humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral


the heart and lungs have xxxx xxxx in the interval both lungs are clear and expanded right middle lobe calcified granuloma is unchanged heart and mediastinum unchanged no change hiatus hernia
=====================================
humeral sequela sequela sequela denser denser denser denser denser denser denser sequela denser humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral


the lungs are clear the cardiomediastinal silhouette is within normal limits no pneumothorax or pleural effusion
=====================================
humeral sequela sequela sequela denser denser denser denser denser denser denser denser sequela denser humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral


frontal and lateral views of the chest show an unchanged cardiomediastinal silhouette normal mediastinal contour pulmonary xxxx and vasculature central airways and aeration of the lungs no focal airspace consolidation or pleural effusion there is subphrenic intraperitoneal extraluminal xxxx free xxxx
=====================================
humeral sequela sequela sequela denser denser denser denser denser denser denser sequela denser humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral humeral
"""

In [ ]:
# evaluate_from_dataloader(val_dataloader,tok,encoder_model,decoder_model,max_len)

In [ ]:
import gdown

url = 'https://drive.google.com/uc?id=1JsLdSbICpbZLJFYn-kWihTKAczj3EeaW'

output = 'pretrainedDense.h5'

gdown.download(url, output, quiet=False)

In [ ]:
pretrainedDense = tf.keras.models.load_model('pretrainedDense.h5',compile=False)
pretrainedDense.summary()

In [ ]:
preDense = tf.keras.models.Model(inputs = pretrainedDense.input, outputs = pretrainedDense.get_layer('decoder').output)
preDense.summary()

In [ ]:
## Input layers
img_input = layers.Input(shape= (256,256,3)) 
report_input= layers.Input(shape= (max_len-1,))

## Encoder ######################

Dense_model = preDense

number_of_encoder_layers=  len(Dense_model.layers)

encoder_output = Dense_model(img_input)
encoder_output = layers.Flatten()(encoder_output)

X_img = layers.Dropout(0.5)(encoder_output)
X_img = layers.Dense(300, use_bias = False, 
                        kernel_regularizer=regularizers.l2(1e-4),
                        name = 'dense_img')(X_img)
X_img = layers.BatchNormalization(name='batch_normalization_img')(X_img)
X_img = layers.Lambda(lambda x : K.expand_dims(x, axis=1))(X_img)

##decoder ########################

X_text = layers.Embedding(vocab_size, 300, mask_zero = True, name = 'emb_text')(report_input)
X_text = layers.Dropout(0.5)(X_text)

# Initial States


LSTMLayer = layers.LSTM(300, return_sequences = True, return_state = True, dropout=0.5, name = 'lstm')

# Take image embedding as the first input to LSTM
_, a, c = LSTMLayer(X_img)

A, _, _ = LSTMLayer(X_text, initial_state=[a, c])
output = layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax',
                                 kernel_regularizer = regularizers.l2(1e-4), 
                                 bias_regularizer = regularizers.l2(1e-4)), name = 'time_distributed_softmax')(A)




model  = Model(inputs=[img_input, report_input], outputs=output, name='NIC_greedy_inference_v2')


##Inference models ################

#encoder_inference model
encoder_model = Model(img_input,[a,c])

# Decoder model ###################

a0 = layers.Input(shape=(300,))
c0 = layers.Input(shape=(300,))

A, alast, clast = LSTMLayer(X_text, initial_state=[a0, c0])
output = layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax',
                                 kernel_regularizer = regularizers.l2(1e-4), 
                                 bias_regularizer = regularizers.l2(1e-4)), name = 'time_distributed_softmax')(A)



decoder_model = Model([report_input,a0,c0],[output,alast,clast])

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes=True)

In [ ]:
epochs =50
lr=1e-4
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr))

In [ ]:
def calls(model_name):
  checkpoint = tf.keras.callbacks.ModelCheckpoint(model_name,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode='min',
                             period=1)
  
  early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                        min_delta=0,
                        patience=20,
                        verbose=1,
                        mode='min')
  
  class myCallBack(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if (logs.get('val_accuracy') > 0.998):
        print ('\nReached 0.998 Validation accuracy!')
        self.model.stop_training = True

  my_call = myCallBack()

  reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10,
                              verbose=1, mode='auto', min_delta=0,
                              cooldown=0, min_lr=0)
  
  lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-5 * 10**(epoch / 10))
  
  return [checkpoint, early, lr_schedule]

In [ ]:
monitor = calls('PreTrainedDenseModel.h5')

In [ ]:
hist = model.fit_generator( train_dataloader,
                            validation_data = val_dataloader,
                            epochs = epochs,
                            callbacks=monitor
                )

In [ ]:
y= hist.history['loss']
y_val = hist.history['val_loss']

x = range(0,len(y))
plt.plot(x,y)
plt.plot(x,y_val)
plt.show()ss

In [ ]:
from MultiCheXNet.evaluation.report_gen_evaluation import get_predictions_from_data_loader
from copy import deepcopy

val_dataloader_tmp = deepcopy(val_dataloader)
val_dataloader_tmp.nb_iteration  = 3
GT , preds = get_predictions_from_data_loader(val_dataloader_tmp,tok,encoder_model, decoder_model,max_len,decoder_type='LSTM')

In [ ]:
for index in range(8):
    print(GT[index])
    print(("====================================="))
    print(preds[index])
    print("")
    print("")

In [ ]:
evaluate_from_dataloader(val_dataloader,tok,encoder_model,decoder_model,max_len)

In [ ]:
"""
(0.014767305709442343,
 0.12152080360762244,
 0.2823516823064442,
 0.34859834137244894)
 """

"""
lungs are overall hyperexpanded with flattening of the diaphragms lungs are clear without focal airspace disease no pleural effusions or pneumothoraces heart and mediastinum of normal size and contour degenerative changes within the spine there are expansile changes within the right clavicle which were seen on the previous xxxxct findings are consistent with changes of multiple myeloma
=====================================
old old exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas


within the posterior lateral th rib there is a deformity along the cortex with associated oblique lucency in addition within the posterior lateral th rib there appears to be a obliquely oriented lucency with cortical disruption findings are concerning for possible left rib fractures otherwise the cardiomediastinal silhouette is within normal limits the lungs are clear bilaterally multiple small punctate radiopaque foreign bodies are seen within the subcutaneous tissues and are present on previous ct scan from xxxx
=====================================
old old exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas


heart size and mediastinal contour within normal limits calcified granuloma right midlung no focal airspace consolidation pneumothorax or large pleural effusion degenerative changes in the thoracic spine
=====================================
old old exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas


there is mild cardiomegaly the transverse xxxx is calcified there is a moderate hiatal hernia the lungs are clear without focal infiltrate no pleural effusion or pneumothorax degenerative changes of the thoracic spine are noted
=====================================
old old exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas


the lungs are clear the heart and pulmonary xxxx are normal the pleural spaces are clear mediastinal contours are normal bony overlap in the lung apices could obscure a small pulmonary nodule
=====================================
old old exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas


the heart pulmonary xxxx and mediastinum are within normal limits there is no pleural effusion or pneumothorax there is no focal air space opacity to suggest a pneumonia both clavicles appear within normal limits on this limited radiographic evaluation
=====================================
old old exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas


heart size and pulmonary vascularity appear within normal limits the lungs are free of focal airspace disease no pleural effusion or pneumothorax is seen
=====================================
old old exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas


the heart and lungs have xxxx xxxx in the interval both lungs are clear and expanded heart and mediastinum normal
=====================================
old old exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams exams granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas granulomas

"""